In [1]:
import pandas as pd

matches = pd.read_csv("matches.csv", index_col=0)
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal


In [2]:
matches = matches.drop(columns=['comp', 'notes', 'season'])

In [3]:
matches["date"] = pd.to_datetime(matches["date"])
#matches["target"] = (matches["result"] == "W").astype("int")
matches["target"] = matches["result"].astype("category").cat.codes
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

matches.head(10)

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,dist,fk,pk,pkatt,team,target,venue_code,opp_code,hour,day_code
1,2023-08-12,12:30,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,0.8,...,19.1,0.0,0,0,Arsenal,2,1,15,12,5
2,2023-08-21,20:00,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,2.0,...,16.4,0.0,1,1,Arsenal,2,0,7,20,0
3,2023-08-26,15:00,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,3.2,...,13.8,0.0,1,1,Arsenal,0,1,9,15,5
4,2023-09-03,16:30,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,2.3,...,15.0,0.0,0,0,Arsenal,2,1,13,16,6
5,2023-09-17,16:30,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,1.0,...,17.4,0.0,0,0,Arsenal,2,0,8,16,6
7,2023-09-24,14:00,Matchweek 6,Sun,Home,D,2.0,2.0,Tottenham,1.8,...,16.9,0.0,1,1,Arsenal,0,1,17,14,6
9,2023-09-30,15:00,Matchweek 7,Sat,Away,W,4.0,0.0,Bournemouth,3.4,...,15.5,0.0,2,2,Arsenal,2,0,2,15,5
11,2023-10-08,16:30,Matchweek 8,Sun,Home,W,1.0,0.0,Manchester City,0.4,...,19.2,0.0,0,0,Arsenal,2,1,12,16,6
12,2023-10-21,17:30,Matchweek 9,Sat,Away,D,2.0,2.0,Chelsea,1.0,...,14.3,0.0,0,0,Arsenal,0,0,6,17,5
14,2023-10-28,15:00,Matchweek 10,Sat,Home,W,5.0,0.0,Sheffield Utd,2.8,...,17.0,0.0,1,1,Arsenal,2,1,16,15,5


In [4]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

train = matches[matches["date"] < '2024-01-01']
test = matches[matches["date"] > '2024-01-01']

In [5]:
train.describe()

,gf,ga,xg,xga,poss,attendance,sh,sot,dist,fk,pk,pkatt,target,venue_code,opp_code,hour,day_code
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,1.556122,1.556122,1.506122,1.506122,50.000000,38291.897959,13.165816,4.418367,16.740306,0.359694,0.130102,0.142857,1.224490,0.500000,9.505102,15.688776,4.704082
std,1.276149,1.276149,0.838226,0.838226,13.819627,17727.024935,5.622894,2.519178,3.022492,0.559236,0.358901,0.371628,0.736767,0.500639,5.781616,2.225193,1.511823
min,0.000000,0.000000,0.000000,0.000000,20.000000,10290.000000,2.000000,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000
25%,1.000000,1.000000,0.900000,0.900000,39.000000,24444.750000,9.000000,3.000000,14.900000,0.000000,0.000000,0.000000,1.000000,0.000000,5.000000,14.000000,5.000000
50%,1.000000,1.000000,1.300000,1.300000,50.000000,31812.000000,12.000000,4.000000,16.700000,0.000000,0.000000,0.000000,1.000000,0.500000,9.000000,15.000000,5.000000
75%,2.000000,2.000000,2.100000,2.100000,61.000000,53117.250000,16.000000,6.000000,18.325000,1.000000,0.000000,0.000000,2.000000,1.000000,15.000000,17.000000,6.000000
max,8.000000,8.000000,4.100000,4.100000,80.000000,73607.000000,34.000000,15.000000,39.900000,2.000000,2.000000,2.000000,2.000000,1.000000,19.000000,20.000000,6.000000


In [6]:
test.describe()

,gf,ga,xg,xga,poss,attendance,sh,sot,dist,fk,pk,pkatt,target,venue_code,opp_code,hour,day_code
count,320.00000,320.00000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.00000,320.000000,320.000000,320.00000,320.000000
mean,1.71875,1.71875,1.568437,1.568437,50.000000,38533.543750,14.156250,4.893750,16.753750,0.384375,0.128125,0.143750,1.08750,0.500000,9.450000,16.20625,4.318750
std,1.40162,1.40162,0.892377,0.892377,13.884205,18081.797256,6.335992,2.716712,2.640625,0.627784,0.370321,0.385422,0.79487,0.500783,5.794603,2.41494,1.824157
min,0.00000,0.00000,0.100000,0.100000,18.000000,10994.000000,2.000000,0.000000,8.400000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,12.00000,0.000000
25%,1.00000,1.00000,0.900000,0.900000,40.000000,24417.000000,9.000000,3.000000,14.900000,0.000000,0.000000,0.000000,0.00000,0.000000,4.000000,15.00000,3.750000
50%,1.00000,1.00000,1.400000,1.400000,50.000000,38193.000000,13.000000,5.000000,16.750000,0.000000,0.000000,0.000000,1.00000,0.500000,9.000000,15.00000,5.000000
75%,3.00000,3.00000,2.100000,2.100000,60.000000,53445.000000,18.000000,6.000000,18.500000,1.000000,0.000000,0.000000,2.00000,1.000000,14.250000,19.00000,5.000000
max,6.00000,6.00000,4.200000,4.200000,82.000000,73612.000000,36.000000,15.000000,25.400000,3.000000,2.000000,2.000000,2.00000,1.000000,19.000000,20.00000,6.000000


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

lr = LogisticRegression(max_iter=1000,
                        random_state=1)

lr.fit(train[predictors], train['target'])

LogisticRegression(max_iter=1000, random_state=1)

In [8]:
preds = lr.predict(test[predictors])

In [9]:
error = accuracy_score(test['target'], preds)

In [10]:
error

0.44375

In [11]:
precision_score(test['target'], preds, 
                average='macro')

C:\Users\Hoang Hiep\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.29583333333333334

In [12]:
precision_score(test['target'], preds, 
                average='micro')

0.44375

In [13]:
precision_score(test['target'], preds, 
                average='weighted')

C:\Users\Hoang Hiep\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.32171874999999994

In [14]:
lr.score(test[predictors], test["target"])

0.44375

In [15]:
combined = pd.DataFrame(dict(actual=test['target'], predicted=preds))

In [16]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,1,2
actual,,
0,44,44
1,71,45
2,45,71


In [17]:
#grouped_matches = matches.groupby("team")
#group = grouped_matches.get_group("Manchester City").sort_values("date")

def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(5, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

#rolling_averages(group, cols, new_cols)

matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')

In [18]:
matches_rolling.index = range(matches_rolling.shape[0])

In [19]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2024-01-01']
    test = data[data["date"] > '2024-01-01']
    lr.fit(train[predictors], train["target"])
    preds = lr.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds, average='micro')
    model_score = lr.score(test[predictors], test["target"])
    return combined, error, model_score

combined, error, model_score = make_predictions(matches_rolling, predictors + new_cols)

In [20]:
combined.head(20)

,actual,predicted
15,2,2
16,2,2
17,2,2
18,2,2
19,2,2
20,2,2
21,2,2
22,2,2
23,0,0
24,2,2


In [21]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1,2
actual,,,
0,7,40,41
1,5,59,52
2,5,29,82


In [22]:
error

0.4625

In [23]:
model_score

0.4625

In [24]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton",
              "Manchester United": "Manchester Utd",
              "Newcastle United": "Newcastle Utd",
              "Nottingham Forest": "Nott'ham Forest",
              "Tottenham Hotspur": "Tottenham",
              "West Ham United": "West Ham",
              "Wolverhampton Wanderers": "Wolves",
              "Sheffield United": "Sheffield Utd"} 

mapping = MissingDict(**map_values)

combined["new_team"] = combined["team"].map(mapping)

In [25]:
combined

,actual,predicted,date,team,opponent,result,new_team
15,2,2,2024-01-20,Arsenal,Crystal Palace,W,Arsenal
16,2,2,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal
17,2,2,2024-02-04,Arsenal,Liverpool,W,Arsenal
18,2,2,2024-02-11,Arsenal,West Ham,W,Arsenal
19,2,2,2024-02-17,Arsenal,Burnley,W,Arsenal
...,...,...,...,...,...,...,...
609,0,1,2024-04-13,Wolverhampton Wanderers,Nott'ham Forest,D,Wolves
610,1,0,2024-04-20,Wolverhampton Wanderers,Arsenal,L,Wolves
611,1,2,2024-04-24,Wolverhampton Wanderers,Bournemouth,L,Wolves
612,2,2,2024-04-27,Wolverhampton Wanderers,Luton Town,W,Wolves


In [26]:
merged = combined.merge(combined, left_on = ["date", "new_team"], right_on = ["date", "opponent"])

merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,2,2,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,1,1,Crystal Palace,Arsenal,L,Crystal Palace
1,2,2,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal,1,1,Nottingham Forest,Arsenal,L,Nott'ham Forest
2,2,2,2024-02-04,Arsenal,Liverpool,W,Arsenal,1,0,Liverpool,Arsenal,L,Liverpool
3,2,2,2024-02-11,Arsenal,West Ham,W,Arsenal,1,1,West Ham United,Arsenal,L,West Ham
4,2,2,2024-02-17,Arsenal,Burnley,W,Arsenal,1,2,Burnley,Arsenal,L,Burnley
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0,1,2024-04-13,Wolverhampton Wanderers,Nott'ham Forest,D,Wolves,0,2,Nottingham Forest,Wolves,D,Nott'ham Forest
316,1,0,2024-04-20,Wolverhampton Wanderers,Arsenal,L,Wolves,2,0,Arsenal,Wolves,W,Arsenal
317,1,2,2024-04-24,Wolverhampton Wanderers,Bournemouth,L,Wolves,2,1,Bournemouth,Wolves,W,Bournemouth
318,2,2,2024-04-27,Wolverhampton Wanderers,Luton Town,W,Wolves,1,1,Luton Town,Wolves,L,Luton Town


In [27]:
merged[(merged["predicted_x"] == 2) & (merged["predicted_y"] == 1)]["actual_x"].value_counts()

2    55
0    30
1    24
Name: actual_x, dtype: int64

In [28]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 2)]["actual_x"].value_counts()

1    55
0    30
2    24
Name: actual_x, dtype: int64

In [29]:
merged[(merged["predicted_x"] == 0) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

0    4
2    2
1    2
Name: actual_x, dtype: int64

In [30]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

2    1
Name: actual_x, dtype: int64

In [31]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 1)]["actual_x"].value_counts()

0    10
2     4
1     4
Name: actual_x, dtype: int64

In [32]:
merged[(merged["predicted_x"] == 2) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

1    3
0    3
2    2
Name: actual_x, dtype: int64

In [33]:
merged[(merged["predicted_x"] == 2) & (merged["predicted_y"] == 2)]["actual_x"].value_counts()

2    25
1    25
0     8
Name: actual_x, dtype: int64